# マンガデータの基礎分析

データ可視化の前に、以下のマンガデータの **基礎分析** を行いましょう。

- `cm_ce.csv`：マンガ各話についてまとめたファイル
- `cm_cc_crt.csv`：マンガ作品とマンガ作者の関係をまとめたファイル

また、各データの作成方法に関しては[マンガデータの前処理](../appendix/cm_preprocess.ipynb)を参照ください。

(basic-analysis)=
## 基礎分析とは

### 一般的な基礎分析

基礎分析とは、データの理解を深め、解析の方向性を定めることを目的とする分析です。
具体的にどのような手順でどのような分析を行うかは、分析の目的やデータの特性に依存するため、一般化することが難しいです。

ですが、Pythonによる基礎分析ツールとして広く用いられているYData Profiling{cite}`ydata`[^4.6]が提供する機能を確認することで、その概観を掴むことは可能です。

[^4.6]: 本書執筆時点の最新版である、v4.6を想定しています。

1. [概要](https://docs.profiling.ydata.ai/4.6/getting-started/concepts/#preview-data)および[品質](https://docs.profiling.ydata.ai/4.6/getting-started/concepts/#data-quality-alerts)の確認
    - データセットの最初の数行を示し、データの構造・列名・タイプを確認
    - データセットの品質の問題を警告する機能。具体的には、列に含まれる数値のパターンが少ない、列に含まれる値が全てゼロ、既定値以上の相関を持つ複数の列が存在する、規定値以上のユニークな数値が存在する、数値が不均衡である、規定値以上の歪度が見られる、欠損値が存在する、無限値が存在する、数値に季節性が見られる、非定常な時系列である、日付あるいは日時に関するレコードが含まれる、数値が一様分布に従っている、文字列・日付・日時列のエントリーの長さがすべて同じ、サポート外の型である、既定値以上の重複が存在する、等
2. [一変数に対する分析](https://docs.profiling.ydata.ai/4.6/getting-started/concepts/#univariate-profiling)
    - 個々の変数に関する基本的な分析結果を提供。具体的にはデータタイプ、欠損値、ユニークな値、基本的な記述統計、そして可視化（ヒストグラムや密度プロット）による分布情報等
3. [多変数に対する分析](https://docs.profiling.ydata.ai/4.6/getting-started/concepts/#multivariate-profiling)
    - 変数間の関係に関する基本的な分析結果を提供。具体的には、相関行列の可視化等。
4. [欠損値の提示](https://docs.profiling.ydata.ai/4.6/getting-started/concepts/#missing-data)
    - 各変数の欠損値の割合を示す。棒グラフやヒートマップ等により、欠損データのパターンを視覚的に表示
5. [外れ値の提示](https://docs.profiling.ydata.ai/4.6/getting-started/concepts/#outliers)
    - 各変数の外れ値の情報を提示する。外れ値の検出には、各変数の分布情報を用いる

### 本書における基礎分析

大前提として、本書では **データ可視化の前段階として** 、データ理解を目的に基礎分析を行います。
よって、上記1-5に含まれるあらゆる可視化はスコープ外です。
また、3に関しては、相関分析に関する高度な数理的知識が必要となるため、基礎分析のスコープ外です。
さらに、5に関しては、分布に関する可視化手法と合わせて分析した方が理解しやすいため、基礎分析では取り扱いません。

以上から、本書における基礎分析では以下を実施します。
説明の都合上、1-5を再編成していることにご注意ください。

- **全体像の把握**：
    - データのサイズやカラム、先頭や末尾のデータの確認
    - 平均、中央値、標準偏差などの基礎統計量を確認し、データの傾向を掴む
    - データに欠損や重複があるか、ある場合はその原因や対処方法を考える
- **各カラムの深掘り**：
    - カラムごとに様々な観点で集計し、その特徴を掴む

最後の「各カラムの深掘り分析」は、筆者が経験に基づき追加した項目です。
「データの全体像の把握」を **広く浅い** 基礎分析とすると、
「各カラムの深掘り」は　**狭く深い** 基礎分析と捉えることができます。
前者に関してはデータに依存しない作業ですが、
後者に関してはデータに合わせてカスタマイズする必要がある作業です。
この二つを組合せることで、データに対して奥行きのある理解を得ることができると信じています。

以下では、実例を通して具体的な作業内容を紹介します。


## 初期設定

以降では、マンガ・アニメ・ゲームデータを可視化するための初期設定を行います。 なお、紙幅の都合のため、書籍版と一部構成が異なることにご注意ください。

### Import

必要なライブラリをImportします。

In [4]:
# warningsモジュールのインポート
import warnings

# データ解析や機械学習のライブラリ使用時の警告を非表示にする目的で警告を無視
# 本書の文脈では、可視化の学習に議論を集中させるために選択した
# ただし、学習以外の場面で、警告を無視する設定は推奨しない
warnings.filterwarnings("ignore")

In [5]:
# osモジュールのインポート
# オペレーティングシステムとのインターフェースを提供
import os

# pathlibモジュールのインポート
# ファイルシステムのパスを扱う
from pathlib import Path

# numpy：数値計算ライブラリのインポート
# npという名前で参照可能
import numpy as np

# pandas：データ解析ライブラリのインポート
# pdという名前で参照可能
import pandas as pd

# tqdm_notebookのインポート
# Jupyter Notebook環境でのプログレスバー表示に使用
# tqdmという名前で参照可能
from tqdm import tqdm_notebook as tqdm


### 定数

本Notebookで用いる定数を定義します。

(python-const)=
:::{admonition} Pythonにおける「定数」
:class: note
一般にプログラミング言語において「定数」とは、プログラムの実行中にその値が変わらないものを指します。
ただし、Pythonには他の言語のような明確な「定数」を定義する構文や機能は存在しません。
慣習的に、大文字の変数を用いて定数であることを明記することが一般的です。
繰り返しになりますが、Pythonにおいては「定数」の値を固定することができない（代入により書き換えることができる）ことに注意してください。
:::

In [6]:
# 読み込み対象ディレクトリの定義

# マンガデータ保存ディレクトリのパス
DIR_IN = Path("../../data/cm/input")

In [7]:
# 読み込み対象ファイル名の定義

# Comic Episode関連のファイル名
FN_CE = "cm_ce.csv"

# Comic CollectionとCReaTor関連のファイル名
FN_CC_CRT = "cm_cc_crt.csv"

## `cm_ce.csv`の基礎分析

`cm_ce.csv`は、マンガ各話に関するテーブル形式のデータです。次のような列を持ちます：

- `ceid`：Comic Episode ID。各話ID
- `cename`：Comic Episode NAME。各話タイトル名
- `ccid`：Comic Collection ID。マンガ作品ID
- `miid`：Magazine Item ID。雑誌巻号ID
- `page_start`：当該各話が開始するページ数
- `page_end`：当該各話が終了するページ数
- `pages`：当該各話の合計ページ数
- `page_start_position`：当該各話の`page_start`の、当該雑誌巻号における位置づけ。0に近いほど雑誌先頭付近に掲載され、1に近いほど雑誌巻末に掲載されていることを表す
- `two_colored`：当該各話に2色カラーページが含まれていたか否かを表す
- `four_colored`：当該各話に4色カラーページが含まれていたか否かを表す
- `miname`：Magazine Item Name。雑誌巻号名
- `mcid`：Magazine Collection ID。雑誌ID
- `mcname`：Magazine Collection NAME。雑誌名
- `date`：発行日
- `price`：価格
- `ccname`：Comic Collection NAME。マンガ作品名

In [8]:
# pandasのread_csv関数でCSVファイルの読み込み
df_ce = pd.read_csv(DIR_IN / FN_CE)

### 全体像の把握

まず、`df_ce`を概観してみましょう。

In [9]:
# df_ceデータフレームの先頭5行を表示し、Tで転置
df_ce.head().T

,0,1,2,3,4
ceid,CE00000,CE00001,CE00002,CE00003,CE00004
cename,第238話/この世代,#134 話の続き,第5話 チア・ザ・マシンガン!,第233話 妖精の輝き,-BOUT 71- From Dark Zone
ccid,C90829,C90482,C90297,C89978,C89929
miid,M535428,M535428,M535428,M535428,M535428
page_start,10.0,33.0,51.0,69.0,89.0
page_end,31.0,50.0,68.0,88.0,108.0
pages,22.0,18.0,18.0,20.0,20.0
page_start_position,0.021368,0.070513,0.108974,0.147436,0.190171
two_colored,False,False,False,False,False
four_colored,True,False,False,False,False


形状を確認してみましょう。

In [10]:
# df_ceデータフレームの形状（行数・列数）を取得
# shape属性を使用すると、(行数, 列数)の形で結果が返される
df_ce.shape

(180076, 16)

欠損値の合計数と、全体に占める割合を確認してみましょう。

In [11]:
# df_ceデータフレーム内の欠損値（NaN）の情報を集計
# isna()メソッドを使用して欠損値の場所をTrueとして特定
# agg()メソッドを使用して、各列における欠損値の合計と平均を計算し、Tで転置して表示
df_ce.isna().agg([sum, "mean"]).T

,sum,mean
ceid,0.0,0.000000
cename,26815.0,0.148909
ccid,9.0,0.000050
miid,0.0,0.000000
page_start,0.0,0.000000
page_end,0.0,0.000000
pages,0.0,0.000000
page_start_position,0.0,0.000000
two_colored,0.0,0.000000
four_colored,0.0,0.000000


`cename`および`price`に一定程度の欠損が見られます。

記述統計情報を確認してみましょう。

In [12]:
# df_ceデータフレームの記述統計情報を取得
# describe()メソッドを使用して、各列の中央値、平均、標準偏差などの基本的な統計量を表示
df_ce.describe()

,page_start,page_end,pages,page_start_position,price
count,180076.000000,180076.000000,180076.000000,180076.000000,180038.000000
mean,210.867847,228.392754,18.524906,0.514882,203.907742
std,123.865626,122.043463,7.713260,0.283149,41.958118
min,1.000000,1.000000,1.000000,0.002045,80.000000
25%,107.000000,126.000000,17.000000,0.274943,180.000000
50%,205.000000,222.000000,19.000000,0.520642,210.000000
75%,305.000000,322.000000,20.000000,0.759642,236.000000
max,581.000000,600.000000,487.000000,1.000000,371.000000


ユニークな値の数を確認してみましょう。

In [13]:
# df_ceデータフレームの各列に対してユニークな値の数をカウント
# nunique()メソッドを使用して、各列のユニークな値の数を計算
# その後、結果を新しいデータフレームとして整形し、列名を"nunique"とする
df_ce.nunique().reset_index(name="nunique")

,index,nunique
0,ceid,180076
1,cename,150446
2,ccid,6953
3,miid,9242
4,page_start,550
5,page_end,568
6,pages,121
7,page_start_position,45279
8,two_colored,2
9,four_colored,2


`ceid`、すなわちマンガ各話IDを主キーとするデータであることがわかります。以降、少し踏み込んだ分析を行います。

### `date`列の深掘り

まず、`date`（発行日）について分析してみましょう。

In [14]:
# df_ceデータフレームの"date"列から最も古い日付と最も新しい日付を取得
# min()メソッドを使用して、最も古い日付を抽出
# max()メソッドを使用して、最も新しい日付を抽出
df_ce["date"].min(), df_ce["date"].max()

('1970-07-27', '2017-07-31')

このデータは1970年7月27日から2017年7月31日までに掲載されたマンガ雑誌を対象としていることがわかります。

`date`をもとに、年（`year`）、月（`month`）、曜日（`weekday`）に関する列を追加します。

In [15]:
# "date"列を日付型に変換
# pd.to_datetime()メソッドを使用して、文字列型の日付を日付型(datetime)に変換
df_ce["date"] = pd.to_datetime(df_ce["date"])

# 年に関する情報を新しい"year"列として追加
# dt.yearを使用して、日付から年のみを取得
df_ce["year"] = df_ce["date"].dt.year

# 月に関する情報を新しい"month"列として追加
# dt.monthを使用して、日付から月のみを取得
df_ce["month"] = df_ce["date"].dt.month

# 曜日に関する情報を新しい"weekday"列として追加
# dt.weekdayを使用して、日付から曜日の情報を数値(0:月曜日, 6:日曜日)として取得
df_ce["weekday"] = df_ce["date"].dt.weekday

年を基準に、各カラムのユニーク数を集計してみましょう。

In [16]:
# "year"（年）ごとに各カラムのユニークな値の数を集計
# groupbyメソッドで"year"を基準にグループ化し、nuniqueメソッドを使用して各カラムのユニークな値の数を計算
# その後、reset_indexメソッドでインデックスをリセットし、データフレームとして結果を返す
df_ce.groupby("year")[["month", "mcid", "miid", "ccid", "ceid"]].nunique().reset_index()

,year,month,mcid,miid,ccid,ceid
0,1970,6,4,86,182,1201
1,1971,12,4,204,295,2746
2,1972,12,4,204,245,2740
3,1973,12,4,205,262,2828
4,1974,12,4,199,188,2387
5,1975,12,4,202,164,2477
6,1976,12,4,200,190,2765
7,1977,12,4,202,194,2839
8,1978,12,4,202,180,2843
9,1979,12,4,200,197,3007


集計開始終了年（`1970`、`2017`）は`month`数が少ないため、掲載データ数が比較的少ないことがわかります。

In [17]:
# "weekday"（曜日）ごとに各カラムのユニークな値の数を集計
# groupbyメソッドで"weekday"を基準にグループ化し、nuniqueメソッドを使用して各カラムのユニークな値の数を計算
# その後、reset_indexメソッドでインデックスをリセットし、データフレームとして結果を返す
df_ce.groupby("weekday")[["mcid", "miid", "ccid", "ceid"]].nunique().reset_index()

,weekday,mcid,miid,ccid,ceid
0,0,4,2751,2777,48901
1,1,4,56,570,1148
2,2,4,3409,2467,74516
3,3,4,1368,1572,30737
4,4,4,508,888,8954
5,5,4,55,530,1110
6,6,4,1095,1306,14710


Pandasにおける`weekday`は`0`が月曜を表し、`6`が日曜を表します。
この集計により、（少なくともこのデータ上では）全てのマンガ雑誌が全ての曜日に発売された実績があることがわかります。
また、水曜（`weekday = 2`）に最も多くの巻号が発売されていたことがわかります。

### `mcid`、`mcname`列の深掘り

`mcid`（マンガ雑誌ID）と`mcname`（マンガ雑誌名）について分析してみましょう。

まず、`mcid`と`mcname`の対応関係を確認します。

In [18]:
# "mcid"（マンガ雑誌ID）ごとにユニークな"mcname"（マンガ雑誌名）の数を集計
# groupbyメソッドで"mcid"を基準にグループ化し、nuniqueメソッドを使用して"mcname"のユニークな値の数を計算
df_ce.groupby("mcid")["mcname"].nunique().reset_index()

,mcid,mcname
0,C117607,1
1,C119033,1
2,C119459,1
3,C120282,1


`mcid`と`mcname`は一対一で対応していることが確認できます。

In [19]:
# "mcname"（マンガ雑誌名）ごとにユニークな"miid"（マンガ巻号ID）と"ceid"（マンガ各話ID）の数を集計
# groupbyメソッドで"mcname"を基準にグループ化し、nuniqueメソッドを使用してそれぞれのユニークな値の数を計算
# sort_valuesメソッドを用いて"ceid"の数で降順にソート
df_ce.groupby("mcname")[["miid", "ceid"]].nunique().sort_values(
    by="ceid", ascending=False
).reset_index()

,mcname,miid,ceid
0,週刊少年サンデー,2307,46172
1,週刊少年チャンピオン,2321,45444
2,週刊少年マガジン,2308,45364
3,週刊少年ジャンプ,2306,43096


このデータは、いわゆる四大少年誌を分析対象としていることがわかります。また、分析対象となるマンガ雑誌巻号（`miid`）数は週刊少年チャンピオンが最も多いですが、各話（`ceid`）数は週刊少年サンデーが最も多いことがわかります。

In [20]:
# "mcname"と"weekday"でグループ化し、ユニークな"miid"の数を集計
df_grouped = df_ce.groupby(["mcname", "weekday"])["miid"].nunique().reset_index()

# 集計結果をマンガ雑誌名x曜日の形式のデータフレームに変形
df_grouped.pivot(index="mcname", columns="weekday", values="miid")

weekday,0,1,2,3,4,5,6
mcname,,,,,,,
週刊少年サンデー,9,9,1706,22,16,15,530
週刊少年ジャンプ,2177,25,25,22,22,15,20
週刊少年チャンピオン,554,9,5,1298,444,5,6
週刊少年マガジン,11,13,1673,26,26,20,539


`週刊少年サンデー`は水曜（`2`)、`週刊少年ジャンプ`は月曜（`0`）、`週刊少年チャンピオン`は木曜（`3`）、そして`週刊少年マガジン`は水曜（`2`）に最も多く発売されていたようです。

### `miid`、`miname`列の深掘り

`miid`（マンガ巻号ID）と`miname`（マンガ巻号名）について分析してみましょう。

まず、`miid`と`miname`の対応関係を確認します。

In [21]:
# "miid"ごとにユニークな"miname"の数を集計し、その統計情報を取得
df_ce.groupby("miid")["miname"].nunique().describe().reset_index()

,index,miname
0,count,9242.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


一つの`miid`に対して、必ず一つの`miname`が対応していることがわかります。

In [22]:
# "miname"ごとにユニークな"miid"の数を集計し、その統計情報を取得
df_ce.groupby("miname")["miid"].nunique().describe().reset_index()

,index,miid
0,count,9226.000000
1,mean,1.001734
2,std,0.041610
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,2.000000


一方で、一つの`miname`に対して、最大で二つの`miid`が対応することがあるようです。

In [23]:
# "miname"ごとにユニークな"miid"の数を集計
df_tmp = df_ce.groupby("miname")["miid"].nunique().reset_index()

# "miid"が複数ある"miname"を抽出
df_tmp[df_tmp["miid"] > 1]

,miname,miid
1556,週刊少年サンデー 2002年 表示号数1,2
1614,週刊少年サンデー 2003年 表示号数2,2
1929,週刊少年サンデー 2009年 表示号数48,2
4346,週刊少年ジャンプ 2012年 表示号数18,2
4365,週刊少年ジャンプ 2012年 表示号数36,2
5496,週刊少年チャンピオン 1988年 表示号数1,2
5982,週刊少年チャンピオン 1998年 表示号数1,2
6270,週刊少年チャンピオン 2004年 表示号数1,2
6510,週刊少年チャンピオン 2009年 表示号数1,2
6672,週刊少年チャンピオン 2012年 表示号数28,2


上記の雑誌巻号については、二つの`miid`が採番されていることがわかります。次に、これらの`miid`に該当する巻号の各話数を見てみます。

In [24]:
# "miid"が複数ある"miname"の一覧を取得
df_multi_miid = df_tmp[df_tmp["miid"] > 1]

# 上記で取得した"miname"に関するデータをフィルタリングし、
# "miid"と"miname"ごとのユニークな"ceid"の数を集計
df_ce[df_ce["miname"].isin(df_multi_miid["miname"].unique())].groupby(
    ["miid", "miname"]
)["ceid"].nunique().reset_index()

,miid,miname,ceid
0,M535787,週刊少年マガジン 2003年 表示号数1,20
1,M535835,週刊少年マガジン 2003年 表示号数1,23
2,M535936,週刊少年マガジン 2000年 表示号数1,22
3,M535984,週刊少年マガジン 2000年 表示号数1,24
4,M536625,週刊少年マガジン 1983年 表示号数14,15
5,M536830,週刊少年マガジン 1983年 表示号数14,14
6,M536985,週刊少年マガジン 1980年 表示号数13,16
7,M536986,週刊少年マガジン 1980年 表示号数13,16
8,M537298,週刊少年マガジン 1973年 表示号数1,11
9,M537349,週刊少年マガジン 1973年 表示号数1,11


`miname`が同じ巻号において、ほとんどの場合は`ceid`を半分ずつ分けて格納しているように見えます。以下については偏りがあります：

- `週刊少年ジャンプ 2012年 表示号数18`：`14`と`9`
- `週刊少年ジャンプ 2012年 表示号数36`：`22`と`1`
- `週刊少年チャンピオン 2012年 表示号数28`：`23`と`2`

例えば`週刊少年ジャンプ 2012年 表示号数18`は以下のようになっています：

In [25]:
# "miname"が"週刊少年ジャンプ 2012年 表示号数18"である行をフィルタリングして取得
df_ce[df_ce["miname"] == "週刊少年ジャンプ 2012年 表示号数18"][
    ["cename", "ccname", "miid", "page_start", "page_end"]
]

,cename,ccname,miid,page_start,page_end
151607,第140箱 「生きることは劇的だ」,めだかボックス,M616417,3.0,37.0
151608,486. The Crimson Cremation,BLEACH,M616417,39.0,55.0
151609,第662話 七武海ローVS.スモーカー中将,ONE PIECE,M616417,57.0,75.0
151610,第227話 トラブル・トラベル-1,SKET DANCE,M616417,77.0,95.0
151611,173ページ 瞬間と最終巻,バクマン。,M616417,97.0,115.0
151612,グルメ 181 食林寺!!,トリコ,M616417,117.0,135.0
151613,Vol.6 泊まりだけど,パジャマな彼女。,M616417,139.0,157.0
151614,第7話 “持たざる者”のことば,ハイキュー!!,M616417,161.0,179.0
151615,第172回 「インフル園児」,いぬまるだしっ,M616417,183.0,191.0
151616,第53話 突撃プロポーズ!!!,magico,M616417,193.0,211.0


この巻号について[詳しく調べてみる](https://mediaarts-db.bunka.go.jp/id/M616417/)と、

> 雑誌を逆さにして、裏表紙も表紙のように活用。表紙から294ページ、裏表・紙から192ページ。

ということがわかりました。よって、`M616417`は表紙側から掲載されている各話、`M616418`は裏表紙側から掲載されている各話であることがわかります。
他のケース全てについて同様の事情があるかはわかりませんが、このデータにおいて、**マンガ雑誌は表紙が複数存在するケースがある**という重要な知見を得られました。

このようなケースは非常に厄介です。一見すると`miname`を基準に`miid`を再度振り直せば解決しそうですが、`page_start`や`page_end`等のページ関連の列は`miid`を基準に作成されていることがあるためです（上記の例では、`M616418`は **背表紙** を基準に各話の位置を算出しているように見えます）。
無理やり`miname`で統合してしまうと他の部分に影響が出る可能性があるため、今回は元データのまま、特にデータの補正を行いませんでした。

(cm-eda-ceid)=
では次は、`miname`に紐付けられているユニークな`ceid`数を集計してみましょう。

In [26]:
# 各"miname"（雑誌巻号名）におけるユニークな"ceid"（各話ID）をカウント
# そして、その掲載話数で昇順にソートする
df_ce.groupby(["miname"])["ceid"].nunique().sort_values().reset_index()

,miname,ceid
0,週刊少年マガジン 1975年 表示号数1,8
1,週刊少年マガジン 1971年 表示号数1,8
2,週刊少年マガジン 1974年 表示号数44,9
3,週刊少年マガジン 1974年 表示号数33,9
4,週刊少年マガジン 1974年 表示号数34,9
...,...,...
9221,週刊少年チャンピオン 2004年 表示号数1,46
9222,週刊少年サンデー 2009年 表示号数48,48
9223,週刊少年チャンピオン 2009年 表示号数1,50
9224,週刊少年サンデー 2013年 表示号数6,50


極端に掲載話数が少ないものと多いものがあることがわかります。発行年ごとに再集計してみます。

In [27]:
# "miname"（雑誌巻号名）ごとに"ceid"（各話ID）のユニーク数と最初の発行年を集計
df_mid = (
    df_ce.groupby("miname")[["ceid", "year"]]
    .agg({"ceid": "nunique", "year": "first"})
    .reset_index()
)

# 発行年（"year"）ごとに、巻号名（"miname"）単位の各話数（"ceid"）の統計情報を集計
df_mid.groupby("year")["ceid"].describe().reset_index()

,year,count,mean,std,min,25%,50%,75%,max
0,1970,86.0,13.965116,2.826129,9.0,12.00,14.0,16.00,23.0
1,1971,204.0,13.460784,2.047089,8.0,12.00,13.0,15.00,26.0
2,1972,204.0,13.431373,1.505391,10.0,12.00,14.0,15.00,16.0
3,1973,204.0,13.862745,1.899127,9.0,13.00,14.0,15.00,22.0
4,1974,199.0,11.994975,1.876213,9.0,10.00,12.0,13.00,17.0
5,1975,202.0,12.262376,1.494757,8.0,11.00,12.0,13.00,19.0
6,1976,200.0,13.825000,1.058289,12.0,13.00,14.0,15.00,16.0
7,1977,202.0,14.054455,0.957819,11.0,13.00,14.0,15.00,17.0
8,1978,202.0,14.074257,1.007154,11.0,13.00,14.0,15.00,18.0
9,1979,200.0,15.035000,1.126999,12.0,14.00,15.0,16.00,19.0


大まかな傾向として、発行年が進むにつれて、一つの`mcname`に紐付けられている`ceid`の数が微増しているようです。

以下では、最も`ceid`数が少ない`miname`と、最も`ceid`数が多い`miname`の中身を確認します。

In [28]:
# 最も`ceid`数が少ない`mcname`（"週刊少年マガジン 1975年 表示号数1"）
# のデータを抽出し、特定のカラムを表示
df_ce[df_ce["miname"] == "週刊少年マガジン 1975年 表示号数1"][
    ["cename", "ccname", "miname", "pages"]
]

,cename,ccname,miname,pages
35089,NaN,うしろの百太郎,週刊少年マガジン 1975年 表示号数1,15.0
35090,〈第三部〉,愛と誠,週刊少年マガジン 1975年 表示号数1,20.0
35091,☆なつかしのスターアルバム その1,天才バカボン,週刊少年マガジン 1975年 表示号数1,12.0
35092,NaN,おれは鉄兵,週刊少年マガジン 1975年 表示号数1,21.0
35093,NaN,イヤハヤ南友,週刊少年マガジン 1975年 表示号数1,18.0
35094,第5章 毛バリの神サマ,釣りキチ三平,週刊少年マガジン 1975年 表示号数1,18.0
35095,第3部-炎の男のバラード,紅の挑戦者,週刊少年マガジン 1975年 表示号数1,19.0
35096,NaN,花の鉄砲松,週刊少年マガジン 1975年 表示号数1,62.0


In [29]:
# 最も`ceid`数が多い`mcname`（"週刊少年ジャンプ 2014年 表示号数4"）
# のデータを抽出し、特定のカラムを表示
df_ce[df_ce["miname"] == "週刊少年ジャンプ 2014年 表示号数4"][
    ["cename", "ccname", "miname", "pages"]
]

,cename,ccname,miname,pages
153472,第73話 大人の時間・二時間目,暗殺教室,週刊少年ジャンプ 2014年 表示号数4,42.0
153473,659:輪墓・辺獄…!!,NARUTO-ナルト-,週刊少年ジャンプ 2014年 表示号数4,17.0
153474,第732話 地下の世界,ONE PIECE,週刊少年ジャンプ 2014年 表示号数4,19.0
153475,グルメ 262 三虎の食卓!!,トリコ,週刊少年ジャンプ 2014年 表示号数4,19.0
153476,第243Q 私の勝ちね,黒子のバスケ,週刊少年ジャンプ 2014年 表示号数4,19.0
153477,第81χ Ψキック・サンタクロース,斉木楠雄のΨ難,週刊少年ジャンプ 2014年 表示号数4,15.0
153478,番外編 夏休みのエリナ,食戟のソーマ,週刊少年ジャンプ 2014年 表示号数4,31.0
153479,第44話 「大規模侵攻」,ワールドトリガー,週刊少年ジャンプ 2014年 表示号数4,19.0
153480,第91話 VS“傘”,ハイキュー!!,週刊少年ジャンプ 2014年 表示号数4,19.0
153481,第4話 VSアシッドルーラー,アイアンナイト,週刊少年ジャンプ 2014年 表示号数4,19.0


`pages`が`1`あるいは`2`の、企画ページまで細かく格納されていることがわかります。
当初は`pages`が一定以下のマンガ各話を除外することを考えましたが、短編の四コママンガ作品を除外することになるため、今回は特に`pages`でフィルターをかけませんでした。

(cm-eda-cc)=
### `ccid`、`ccname`列の深掘り

`ccid`（マンガ作品ID）、`ccname`（マンガ作品名）について分析してみましょう。

まずは、`ccid`と`ccname`のの対応関係を確認します。

In [30]:
# 各`ccid`に関連付けられたユニークな`ccname`の数を集計
# その結果の統計情報（平均、最小値、最大値など）を取得
df_ce.groupby("ccid")["ccname"].nunique().describe().reset_index()

,index,ccname
0,count,6953.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


`ccid`は必ず一つの`ccname`と紐づいているようです。

In [31]:
# 各`ccname`に関連付けられたユニークな`ccid`の数を集計
# その結果の統計情報（平均、最小値、最大値など）を取得
df_ce.groupby("ccname")["ccid"].nunique().describe().reset_index()

,index,ccid
0,count,6845.000000
1,mean,1.015778
2,std,0.179411
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,9.000000


一方で、`ccname`が同一であっても、`ccid`が異なるケースが存在します。
複数の`ccid`と紐づいたものを確認してみましょう。

In [32]:
# 各`ccname`ごとに関連付けられたユニークな`ccid`の数を集計
df_tmp = (
    df_ce.groupby("ccname")["ccid"].nunique().sort_values(ascending=False).reset_index()
)

# `ccid`数が1より多い`ccname`をリストとして取得
ccnames = df_tmp[df_tmp["ccid"] > 1]["ccname"].unique()

# 上記で取得した`ccname`リストを基に、各`ccname`と`ccid`の組み合わせごとに
# ユニークな`ceid`の数と`pages`の合計を集計
df_ce[df_ce["ccname"].isin(ccnames)].groupby(["ccname", "ccid"])[["ceid", "pages"]].agg(
    {"ceid": "nunique", "pages": sum}
).reset_index()

,ccname,ccid,ceid,pages
0,DEATH NOTE,C87653,111,2433.0
1,DEATH NOTE,C87654,1,1.0
2,FAIRY TAIL,C89977,1,8.0
3,FAIRY TAIL,C89978,547,11617.0
4,MAJOR,C91652,292,5912.0
...,...,...,...,...
190,青い鳥の伝説,C94258,35,728.0
191,風,C94019,13,292.0
192,風,C94020,2,81.0
193,食戟のソーマ,C110965,1,7.0


少なくない数の`ccname`に対して、複数の`ccid`が割り当てられていました。
全てを確認することはできないので、先頭20個を抽出してみます。

In [33]:
# 上記で取得した`ccname`リストを基に、各`ccname`と`ccid`の組み合わせごとに
# ユニークな`ceid`の数と`pages`の合計を集計し、その結果の先頭20件を表示
df_ce[df_ce["ccname"].isin(ccnames)].groupby(["ccname", "ccid"])[["ceid", "pages"]].agg(
    {"ceid": "nunique", "pages": sum}
).reset_index().head(20)

,ccname,ccid,ceid,pages
0,DEATH NOTE,C87653,111,2433.0
1,DEATH NOTE,C87654,1,1.0
2,FAIRY TAIL,C89977,1,8.0
3,FAIRY TAIL,C89978,547,11617.0
4,MAJOR,C91652,292,5912.0
5,MAJOR,C91653,456,8413.0
6,STAR WARS,C90231,1,19.0
7,STAR WARS,C90232,5,91.0
8,[イラスト],C88078,1,1.0
9,[イラスト],C90385,1,1.0


ほとんどのケースは、単発の企画ページがたまたま同名の`ccname`であることが確認できます。気になるのは、どちらの`ccid`についてもそれなりのボリュームのデータが存在する`MAJOR`です。

In [34]:
# `ccname`が"MAJOR"であるデータに絞り込み、
# 各`ccid`における`year`の統計情報（count, mean, stdなど）を集計
df_ce[df_ce["ccname"] == "MAJOR"].groupby("ccid")["year"].describe().reset_index()

,ccid,count,mean,std,min,25%,50%,75%,max
0,C91652,292.0,1997.150685,1.808124,1994.0,1996.0,1997.0,1999.0,2000.0
1,C91653,456.0,2005.157895,2.908355,2000.0,2003.0,2005.0,2008.0,2013.0


経緯は不明ですが、`MAJOR`は`2000`年付近を境に`ccid`が変更されているようです。

次に、`ccname`ごとの`ceid`数を集計してみます。

In [35]:
# `ccname`と`mcname`の組み合わせごとに、それぞれの`ceid`のユニーク数（掲載数）を集計
# 結果を`ceid`の数で降順ソートして上位20件を表示
df_ce.groupby(["ccname", "mcname"])["ceid"].nunique().sort_values(
    ascending=False
).reset_index().head(20)

,ccname,mcname,ceid
0,こちら葛飾区亀有公園前派出所,週刊少年ジャンプ,1968
1,はじめの一歩,週刊少年マガジン,1186
2,名探偵コナン,週刊少年サンデー,1009
3,ONE PIECE,週刊少年ジャンプ,893
4,MAJOR,週刊少年サンデー,748
5,NARUTO-ナルト-,週刊少年ジャンプ,722
6,BLEACH,週刊少年ジャンプ,715
7,銀魂,週刊少年ジャンプ,655
8,ドカベン,週刊少年チャンピオン,635
9,ジョジョの奇妙な冒険,週刊少年ジャンプ,594


今回扱うデータ中のマンガ作品のうち、最も各話数が多いのは`こちら葛飾区亀有公園前派出所`であることがわかります。

次に、マンガ作品名の長さについて分析してみます。

In [36]:
# 各`ccid`に対して、その`ccid`を持つ`ccname`、`mcname`、およびユニークな`ceid`の数を取得
df_tmp = (
    df_ce.groupby("ccid")[["ccname", "mcname", "ceid"]]
    .agg(
        {
            "ccname": "first",
            "mcname": "first",
            "ceid": "nunique",
        }
    )
    .reset_index()
)

# さらに、各`ccname`の文字数を`l_ccname`という新しい列に保存
df_tmp["l_ccname"] = df_tmp["ccname"].str.len()

In [37]:
# `mcname`（マンガ雑誌名）ごとに、`ccname`（マンガ作品名）の長さの統計情報を取得
df_tmp.groupby("mcname")["l_ccname"].describe()

,count,mean,std,min,25%,50%,75%,max
mcname,,,,,,,,
週刊少年サンデー,1496.0,7.890374,4.334664,1.0,5.0,7.0,10.0,44.0
週刊少年ジャンプ,2129.0,9.114138,5.262260,1.0,6.0,8.0,11.0,52.0
週刊少年チャンピオン,1736.0,7.964286,4.409564,1.0,5.0,7.0,9.0,52.0
週刊少年マガジン,1592.0,8.615578,4.784136,1.0,6.0,8.0,10.0,41.0


平均的にマンガ作品名が長いのは、`週刊少年ジャンプ`、`週刊少年チャンピオン`、`週刊少年マガジン`、そして`週刊少年サンデー`の順であることがわかりました。
特にマンガ作品名が長いものを見てみます。

In [38]:
# `l_ccname`（マンガ作品名の長さ）を降順にソートし、上位10作品を抽出
# 見やすいように特定の列のみ抽出
df_tmp.sort_values("l_ccname", ascending=False).head(10)[["ccname", "mcname", "ceid"]]

,ccname,mcname,ceid
6169,3大パニックホラー特別コラボマンガ「ハカイジュウ」VS「魔法少女オブ・ジ・エンド」VS「バー...,週刊少年チャンピオン,1
257,「ONE PIECE」公式スピンオフコメディ!!「ワンピースパーティー」JC2巻発売記念SP...,週刊少年ジャンプ,1
6183,死にたくなるしょうもない日々が死にたくなるくらいしょうもなくて死にたくなるほど死にたくない日々,週刊少年チャンピオン,3
5303,映画公開記念!!『真救世主伝説 北斗の拳 ラオウ伝殉愛の章』 武論尊突撃インタビュー!!,週刊少年サンデー,1
4762,杉本ペロ新連載への軌跡 ～栄光のゴールをめざしてRoad to Shinrensai～,週刊少年サンデー,1
1938,データカードダス NARUTO-ナルト-疾風伝 ナルティメットミッション で遊ぼう!!,週刊少年ジャンプ,1
569,パイレーツ・オブ・カリビアン 最後の海賊 ワールドプレミアレポート from 上海,週刊少年マガジン,1
5016,毎日がワイルドアームズ WILD ARMS the 4th Detonator,週刊少年サンデー,4
582,祝!ONE PIECE 20周年!!尾田さんとの思い出漫画! by しまぶー.,週刊少年ジャンプ,1
250,BORUTO-ボルト- -NARUTO NEXT GENERATIONS-,週刊少年ジャンプ,15


`ccname`が長いマンガ作品のほとんどは企画ものであることがわかりました。
そこで、`ceid`が2以上のマンガ作品に絞ってみます。

In [39]:
# `l_ccname`（マンガ作品名の長さ）を降順にソートし、上位10作品を抽出
# 見やすいように特定の列のみ抽出
df_tmp[df_tmp["ceid"] > 1].sort_values("l_ccname", ascending=False).head(10)[
    ["ccname", "mcname", "ceid"]
]

,ccname,mcname,ceid
6183,死にたくなるしょうもない日々が死にたくなるくらいしょうもなくて死にたくなるほど死にたくない日々,週刊少年チャンピオン,3
5016,毎日がワイルドアームズ WILD ARMS the 4th Detonator,週刊少年サンデー,4
250,BORUTO-ボルト- -NARUTO NEXT GENERATIONS-,週刊少年ジャンプ,15
6516,Ping-Pong Panic Paradise 水原先輩がゆく!!,週刊少年チャンピオン,4
5471,3マッスルズコラボ漫画 囚人リク 真・餓狼伝 バチバチBURST,週刊少年チャンピオン,2
1544,攻・守 -KOSHU- MY FRIEND/OUR DREAM,週刊少年ジャンプ,2
164,漫画家志望必見!!先輩漫画家が極意を伝授!! 進め!漫画道!!,週刊少年サンデー,37
165,漫画家志望必見!!先輩漫画家が極意を伝授!! 進め!漫画道!!,週刊少年サンデー,6
990,青き波濤 -『がんばろうKOBE』'95ブルーウェーブの軌跡-,週刊少年ジャンプ,2
203,漫画家志望必見!!先輩漫画家が極意を伝授!! 進め!漫画道!!,週刊少年サンデー,6


このデータ中で最も作品名が長いマンガ作品は、`死にたくなるしょうもない日々が死にたくなるくらいしょうもなくて死にたくなるほど死にたくない日々`であることがわかりました。

(cm-eda-ce)=
### `ceid`、`cename`列の深掘り

`ceid`（マンガ各話ID）、`cename`（マンガ各話名）について分析してみましょう。

まずは、`ceid`と`cename`の対応関係を確認します。

In [40]:
# 各`ceid`に関連付けられたユニークな`cename`の数を集計
# その結果の統計情報（平均、最小値、最大値など）を取得
df_ce.groupby("ceid")["cename"].nunique().describe().reset_index()

,index,cename
0,count,180076.000000
1,mean,0.851091
2,std,0.356000
3,min,0.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,1.000000


`cename`が空白（`None`）の場合があるため、最小値（`min`）が0になっています。それ以外は、一つの`ceid`について一つの`cename`が対応付けられているようです。

In [41]:
# 各`cename`に関連付けられたユニークな`ceid`の数を集計
# その結果の統計情報（平均、最小値、最大値など）を取得
df_ce.groupby("cename")["ceid"].nunique().describe().reset_index()

,index,ceid
0,count,150446.000000
1,mean,1.018711
2,std,0.862405
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,186.000000


最大値（`max`）が想定より多くなってしまいました。具体例を見てみましょう。

In [42]:
# df_ceでcenameごとに異なるceidの数をカウントし、降順ソートした上で上位5つを表示
df_ce.groupby("cename")["ceid"].nunique().sort_values(
    ascending=False
).reset_index().head()

,cename,ceid
0,プロ野球編,186
1,後編,134
2,前編,118
3,最終回,82
4,第三部 孤狼青春編,81


最も重複が多いマンガ各話名は`プロ野球編`であることがわかりました。
どのマンガ作品で用いられた各話名でしょうか？

In [43]:
# cenameがプロ野球編と一致するccnameのレコード数を集計
df_ce[df_ce["cename"] == "プロ野球編"].value_counts("ccname").reset_index()

,ccname,count
0,ドカベン,186


本書で扱うマンガデータにおいては、`ドカベン`に対して合計186話分の`プロ野球編`が対応付けられていることがわかりました。

次に、マンガ作品ごとの各話数を集計し、基礎統計量を算出してみます。

In [44]:
# df_ceでccnameごとに異なるceidの数をカウントし、要約統計量を算出
df_ce.groupby("ccname")["ceid"].nunique().describe().reset_index()

,index,ceid
0,count,6845.000000
1,mean,26.306355
2,std,71.718334
3,min,1.000000
4,25%,1.000000
5,50%,2.000000
6,75%,18.000000
7,max,1968.000000


最小で1話、平均で約26話、最大で1968話のマンガ各話を含むマンガ作品があることがわかりました。
[前述](cm-eda-cc)した通り、1968話は`こちら葛飾区亀有公園前派出所`です。
では、各話数が少ないマンガ作品について分析してみましょう。
まずは、各話数の少ないマンガ作品数を集計します。

In [45]:
# df_ceでccnameごとに異なるceidの数をカウントし、新しいDataFrameに格納
df_tmp = df_ce.groupby("ccname")["ceid"].nunique().reset_index(name="n_ce")

# 新しいDataFrameでn_ce（ユニークなceidの数）ごとに異なるccnameの数をカウントし、上位10の結果を表示
df_tmp.groupby("n_ce")["ccname"].nunique().reset_index().head(10)

,n_ce,ccname
0,1,3344
1,2,530
2,3,208
3,4,150
4,5,126
5,6,61
6,7,45
7,8,67
8,9,47
9,10,95


合計各話数が1の作品が非常に多く、それ以降徐々に減少していることがわかります。
例えば、合計各話数が1の作品としてどのようなものが存在するのでしょうか？

In [46]:
# 合計各話数が1のマンガ作品を抽出
df_tmp[df_tmp["n_ce"] == 1].reset_index(drop=True)

,ccname,n_ce
0,'80オリンピックに燃える日本のホープ!!,1
1,'83打ち上げ!!爆笑ギャグ・バトル,1
2,-SINS-,1
3,-豆次郎くん-MAMEJIR〔O＾＾〕,1
4,006,1
...,...,...
3339,ＧｏＧｏ!合成マン,1
3340,ＪＳＧ杯 4コマ発表,1
3341,ＵＦＯが日本の空を飛ぶ日,1
3342,～HWGP～ 東村山ウエスト・ゲート・パーク,1


このように、合計各話数が小さいマンガ作品の多くは、読切、タイアップ、そして不定期掲載等であることがわかります。

では、 **連載マンガ作品** を識別するにはどうしたら良いでしょうか？
本書では連載マンガ作品を対象とした可視化を多く取り上げますが、データには連載マンガ作品を表すフラグ等はないため、合計各話数を手がかりに判断する必要があります。

短期連載の理由はさまざまですが、どんなに短い連載であっても単行本1冊分の各話数は掲載されると考えるのが自然です。
単純に考えると、単行本1冊のページ数が約200ページ、各話のページ数が約20ページ程度ですので、週刊連載10話で単行本1冊分の分量になります。
ただし、新連載直後は1話あたりのページ数が増えることが多いので、第1巻に限り8-9話のみ収録されていることもあります。
また、上記の集計結果を見ると、合計各話数が10話以下のマンガ作品の中では8話に該当する作品が多いことがわかります。
以上から、本書では便宜上、 **合計各話数が8話以上** の作品を連載マンガ作品とみなします。

(cm-eda-pages)=
### `page_start`、`page_end`、`pages`、`page_start_position`列の深掘り

ページに関するカラムについて分析してみましょう。

- `page_start`：開始ページ
- `page_end`：終了ページ
- `pages`：各話のページ数
- `page_start_position`：巻号内の開始ページの位置

In [47]:
# `page_start`、`page_end`、`pages`、および`page_start_position`列の基礎統計情報を表示
df_ce[["page_start", "page_end", "pages", "page_start_position"]].describe()

,page_start,page_end,pages,page_start_position
count,180076.000000,180076.000000,180076.000000,180076.000000
mean,210.867847,228.392754,18.524906,0.514882
std,123.865626,122.043463,7.713260,0.283149
min,1.000000,1.000000,1.000000,0.002045
25%,107.000000,126.000000,17.000000,0.274943
50%,205.000000,222.000000,19.000000,0.520642
75%,305.000000,322.000000,20.000000,0.759642
max,581.000000,600.000000,487.000000,1.000000


`pages`が非常に大きい各話が存在することが気になります。

In [48]:
# `pages`列の値を降順に並び替えて、上位5行を抽出
# 抽出する列は`cename`、`ccname`、`miname`、および`pages`
df_ce.sort_values("pages", ascending=False)[
    ["cename", "ccname", "miname", "pages"]
].head()

,cename,ccname,miname,pages
168891,第161話 御目出度いのに!で候,磯部磯兵衛物語～浮世はつらいよ～,週刊少年ジャンプ 2015年 表示号数48,487.0
127239,CASE83.ナギの島2,アクシデンツ,週刊少年サンデー 1998年 表示号数14,455.0
19085,社長登場! ／ 面接 ／ サイン,宮田すすむと日本の社長!,週刊少年マガジン 1994年 表示号数24,392.0
171179,第9話 侵入者,アンペア,週刊少年サンデー 2015年 表示号数35,318.0
10386,RAVE:152 裸の英雄!?,RAVE,週刊少年マガジン 2002年 表示号数36,234.0


筆者の記憶では`磯部磯兵衛物語～浮世はつらいよ～`や`RAVE`は巻号一冊中に二話同時に掲載されることがありました。
データ上ではそれらを区別せず、`page_start`は一話目の掲載位置、`page_end`は二話目の掲載位置を記録してしまっているため、`pages`が大きくなってしまったのではないかと推測しています。

### `two_colored`、`four_colored`列の深掘り

`two_colored`（マンガ各話が2色カラーページを含むか否か）と`four_colored`（マンガ各話が4色カラーページを含むか否か）について分析してみましょう。
まず、カラーページを含むマンガ各話の総数とその比率を集計してみます。

In [49]:
# two_colored・four_colored列がTrueとなっている各話の総数と、全体に対する比率を集計
df_ce[["two_colored", "four_colored"]].agg(["sum", "mean"]).T.reset_index()

,index,sum,mean
0,two_colored,6776.0,0.037629
1,four_colored,12926.0,0.071781


2色カラーを含むマンガ各話は合計で`6776`話（全体の約4%）であり、4色カラーを含むマンガ各話は合計で`12926`話（全体の約7%）であることがわかりました。

特に2色カラーについては、本書執筆時点でほとんど見かけません。年代別に合計各話数を集計してみましょう。

In [50]:
# 年ごとにtwo_coloredとfour_coloredの各話数を集計
df_ce.groupby("year")[["two_colored", "four_colored"]].sum().reset_index()

,year,two_colored,four_colored
0,1970,24,130
1,1971,40,280
2,1972,3,296
3,1973,50,232
4,1974,279,187
5,1975,363,172
6,1976,293,210
7,1977,352,185
8,1978,369,167
9,1979,380,183


2色カラーは`1974`年から増え始め、`1984`年にピークを迎え、それ以降は減少し続けていることがわかります。
一方で4色カラーは`1990`年頃に2色カラーを逆転し、それ以降なだらかに増加していることがわかります。

マンガ雑誌別に各カラー各話の合計数を集計してみましょう。

In [51]:
# マンガ雑誌ごとにtwo_color列とfour_color列がTrueとなっている各話の総数を集計
df_ce.groupby("mcname")[["two_colored", "four_colored"]].sum().reset_index()

,mcname,two_colored,four_colored
0,週刊少年サンデー,1753,2490
1,週刊少年ジャンプ,2361,4347
2,週刊少年チャンピオン,1467,3508
3,週刊少年マガジン,1195,2581


2色カラー、4色カラーのいずれも`週刊少年ジャンプ`が最も多い各話数を掲載していることがわかりました。

In [52]:
# マンガ雑誌ごとにtwo_colored/four_coloredcolor列のTrueの総数を集計しdf_tmpとして格納
df_tmp = df_ce.groupby("ccname")[["two_colored", "four_colored"]].sum().reset_index()
# df_tmpをtwo_colored列に関して降順ソートし、上位10作品を表示
df_tmp.sort_values("two_colored", ascending=False, ignore_index=True).head()

,ccname,two_colored,four_colored
0,がきデカ,161,70
1,ドカベン,109,200
2,らんぽう,79,29
3,釣りキチ三平,74,17
4,キン肉マン,72,22


2色カラーの各話数が最も多かったのは`がきデカ`でした。

In [53]:
# df_tmpをfour_colored列に関して降順ソートし、上位10作品を表示
df_tmp.sort_values("four_colored", ascending=False, ignore_index=True).head()

,ccname,two_colored,four_colored
0,ドカベン,109,200
1,ONE PIECE,10,158
2,弱虫ペダル,0,148
3,範馬刃牙 SON OF OGRE,0,135
4,こちら葛飾区亀有公園前派出所,71,124


一方で4色カラーの各話数が最も多かったのは`ドカベン`でした。

### `price`列の深掘り

`price`（雑誌巻号の価格）について分析してみましょう。

In [54]:
# `price`列に関する統計情報を取得
df_ce[["price"]].describe()

,price
count,180038.000000
mean,203.907742
std,41.958118
min,80.000000
25%,180.000000
50%,210.000000
75%,236.000000
max,371.000000


最大値の371円が気になります。
`price`でソートすることで、特に`price`が大きいものと、小さいものを確認してみましょう。

In [55]:
# 各`miname`と`year`の組み合わせごとに最初の`price`を取得し、降順にソート
df_ce.groupby(["miname", "year"])["price"].first().reset_index().sort_values(
    "price", ascending=False
)

,miname,year,price
1835,週刊少年サンデー 2007年 表示号数50,2007,371.0
2196,週刊少年サンデー 2015年 表示号数27,2015,280.0
2205,週刊少年サンデー 2015年 表示号数36,2015,280.0
2214,週刊少年サンデー 2015年 表示号数45,2015,280.0
2215,週刊少年サンデー 2015年 表示号数46,2015,280.0
...,...,...,...
6931,週刊少年マガジン 1970年 表示号数39,1970,80.0
6932,週刊少年マガジン 1970年 表示号数40,1970,80.0
4613,週刊少年チャンピオン 1970年 表示号数19,1970,80.0
2366,週刊少年ジャンプ 1971年 表示号数47,1971,NaN


データ中、最も`price`が大きいものは`週刊少年サンデー 2007年 表示号数50`であることがわかりました。（ただし、この情報の裏付けは取れませんでした。）

年代ごとの価格の推移を集計してみます。
`df_ce`をそのまま集計すると掲載各話数が多い雑誌巻号の価格の影響が大きくなってしまうので、一度雑誌巻号単位のデータに集計した上で集計する必要があることにご注意ください。

In [56]:
# 各`miname`ごとに最初に出現する`year`と`price`を取得
df_tmp = df_ce.groupby("miname")[["year", "price"]].first().reset_index()

# 取得したデータを`year`単位で集計し、`price`に関する統計情報を取得
df_tmp.groupby(["year"])["price"].describe()

,count,mean,std,min,25%,50%,75%,max
year,,,,,,,,
1970,86.0,83.023256,4.619592,80.0,80.0,80.0,90.0,90.0
1971,203.0,87.339901,5.341574,80.0,80.0,90.0,90.0,100.0
1972,204.0,95.392157,5.556972,90.0,90.0,100.0,100.0,120.0
1973,204.0,101.911765,6.715328,90.0,100.0,100.0,100.0,120.0
1974,199.0,128.592965,6.278885,100.0,130.0,130.0,130.0,150.0
1975,202.0,134.702970,8.473599,130.0,130.0,130.0,130.0,150.0
1976,200.0,149.100000,4.504604,130.0,150.0,150.0,150.0,170.0
1977,202.0,156.336634,9.327941,150.0,150.0,150.0,170.0,170.0
1978,202.0,156.732673,9.474649,150.0,150.0,150.0,170.0,170.0


`mean`を見ると、毎年じわじわ`price`が上昇していることがわかります。

## `cm_cc_crt.csv`の基礎分析

`cm_cc_crt.csv`は、マンガ作品とその作者に関するテーブル形式のデータです。次のような列を含みます：

- `ccid`： Comic Collection ID。マンガ作品ID
- `crtid`： CReaTor ID。作者ID
- `ccname`： Comic Collection NAME。マンガ作品名
- `crtname`： CReaTor NAME。作者名
- `mcid`： Magazine Collection ID。雑誌ID
- `mcname`： Magazine Collection NAME。雑誌名
- `n_ce`： Number of Commic Episodes。当該マンガ作品の合計各話数
- `n_2c`: Number of Two Colored episodes。当該マンガ作品の2色カラー話数
- `n_4c`: Number of Four Colored episodes。当該マンガ作品の4色カラー話数
- `first_date`：当該マンガ作品の最初の掲載日
- `last_date`：当該マンガ作品の最後の掲載日

In [57]:
# pandasのread_csv関数でCSVファイルの読み込み
df_cc_crt = pd.read_csv(DIR_IN / FN_CC_CRT)

### 全体像の把握

まず、`df_cc_crt`を概観してみましょう。

In [58]:
# df_cc_crtデータフレームの先頭5行をTで転置して表示
df_cc_crt.head().T

,0,1,2,3,4
ccid,C87429,C87430,C87430,C87431,C87432
crtid,CCRT01604,CCRT02117,CCRT03152,CCRT01415,CCRT01979
ccname,交通安全'76,好敵手 室伏広治物語,好敵手 室伏広治物語,鋼鉄の殺人者,硬派山崎銀次郎
n_ce,1,1,1,1,1
n_2c,0,0,0,0,0
n_4c,0,0,0,0,1
first_date,1976-09-06,2000-10-02,2000-10-02,1979-12-10,1974-04-29
last_date,1976-09-06,2000-10-02,2000-10-02,1979-12-10,1974-04-29
mcid,C119459,C119459,C119459,C119459,C119459
mcname,週刊少年ジャンプ,週刊少年ジャンプ,週刊少年ジャンプ,週刊少年ジャンプ,週刊少年ジャンプ


形状を確認してみましょう。

In [59]:
# df_cc_crtデータフレームの形状（行数・列数）を取得
# shape属性を使用すると、(行数, 列数)の形で結果が返される
df_cc_crt.shape

(7909, 11)

欠損値の合計数と、全体に占める割合を確認してみましょう。

In [60]:
# df_cc_crtデータフレーム内の欠損値（NaN）の情報を集計
# isna()メソッドを使用して欠損値の場所をTrueとして特定
# agg()メソッドを使用して、各列における欠損値の合計と平均を計算し、転置して表示
df_cc_crt.isna().agg([sum, "mean"]).T

,sum,mean
ccid,0.0,0.0
crtid,0.0,0.0
ccname,0.0,0.0
n_ce,0.0,0.0
n_2c,0.0,0.0
n_4c,0.0,0.0
first_date,0.0,0.0
last_date,0.0,0.0
mcid,0.0,0.0
mcname,0.0,0.0


記述統計情報を確認してみましょう。

In [61]:
# df_cc_crtデータフレームの記述統計情報を取得
# describe()メソッドを使用して、各列の中央値、平均、標準偏差などの基本的な統計量を表示
df_cc_crt.describe()

,n_ce,n_2c,n_4c
count,7909.000000,7909.000000,7909.000000
mean,26.584524,1.001391,1.923631
std,69.735811,4.638895,6.850708
min,1.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000
50%,2.000000,0.000000,0.000000
75%,19.000000,0.000000,1.000000
max,1968.000000,161.000000,200.000000


ユニークな値の数を確認してみましょう。

In [62]:
# df_cc_crtデータフレームの各列に対してユニークな値の数をカウント
# nunique()メソッドを使用して、各列のユニークな値の数を計算
# その後、結果を新しいデータフレームとして整形し、列名を"nunique"とする
df_cc_crt.nunique().reset_index(name="nunique")

,index,nunique
0,ccid,6844
1,crtid,3146
2,ccname,6745
3,n_ce,340
4,n_2c,52
5,n_4c,65
6,first_date,4341
7,last_date,4115
8,mcid,4
9,mcname,4


行数より`ccid`および`crtid`のユニーク数が少ないので、それぞれ単独では重複が存在することがわかります。

In [63]:
# `ccid`と`crtid`の組み合わせで重複する行の数をカウント
df_cc_crt.duplicated(subset=["ccid", "crtid"]).sum()

0

`ccid`と`crtid`の組合せに対して重複がないことが確認できます。

`ccid`のカバー率（`cm_ce.csv`の`ccid`のうち、`cm_cc_crt.csv`に存在するものの割合）を確認します。

In [64]:
# `df_cc_crt`と`df_ce`で共通の`ccid`の数を取得
n_ccid_w_crt = len(set(df_cc_crt["ccid"].unique()) & set(df_ce["ccid"].unique()))

# `df_ce`内のユニークな`ccid`の数を取得
n_ccid = df_ce["ccid"].nunique()

# カバー率を計算して表示
print(f"{n_ccid_w_crt} / {n_ccid} = {n_ccid_w_crt / n_ccid}")

6844 / 6953 = 0.9843233136775492


全体の約98%、6844のマンガ作品に対してマンガ作者が対応付けられていることがわかりました。
以降、カラムごとに少し踏み込んだ分析を行います。

### `ccid`、`ccname`列の深掘り

まず、`ccid`（マンガ作品ID）と`ccname`（マンガ作品名）について分析してみましょう。

In [65]:
# `ccid`と`ccname`ごとにユニークな`crtid`の数を集計
# その後、結果の統計情報を取得
df_cc_crt.groupby(["ccid", "ccname"])["crtid"].nunique().describe().reset_index()

,index,crtid
0,count,6844.000000
1,mean,1.155611
2,std,0.449605
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,13.000000


一つのマンガ作品に対して、平均で約1.2人、最大で約13人のマンガ作者が紐付けられていることがわかります。

では、紐付いているマンガ作者が多いマンガ作品を確認してみましょう。

In [66]:
# `ccid`と`ccname`ごとにユニークな`crtid`の数を集計
# その後、降順にソートして、紐づいているマンガ作者が多いマンガ作品を上位に表示
df_cc_crt.groupby(["ccid", "ccname"])["crtid"].nunique().sort_values(
    ascending=False
).reset_index()

,ccid,ccname,crtid
0,C90714,TheThreeWoodCreation殺人事件,13
1,C94429,オールスターギャグ祭り 笑いの力,12
2,C89927,'83打ち上げ!!爆笑ギャグ・バトル,7
3,C93156,ツンツンサワサワ春を待つ,6
4,C89830,両津VS首領パッチ!! Jギャグオールスターズ葛飾大騒動,5
...,...,...,...
6839,C89810,ライジングインパクト,1
6840,C89809,ライオンブックス,1
6841,C89808,ラーマゲドン拉麺最終戦争,1
6842,C89807,四丁目の怪人くん,1


最も多くのマンガ作者と紐付けられているのは、`TheThreeWoodCreation殺人事件`でした。
詳細を確認してみましょう。

In [67]:
# `ccid`が"C90714"であるレコードを抽出し、特定のカラムのみ表示
df_ce[df_ce["ccid"] == "C90714"][["ccname", "miname", "mcname", "date"]]

,ccname,miname,mcname,date
29872,TheThreeWoodCreation殺人事件,週刊少年マガジン 1982年 表示号数18,週刊少年マガジン,1982-04-21


これは、`週刊少年マガジン 1982年 表示号数18`に掲載された、以下の13名のマンガ家による合作作品であることがわかりました。
では、具体的にどのマンガ作者がこの作品に関わっていたのでしょうか？

In [68]:
# `ccid`が"C90714"であるレコードを`df_cc_crt`から抽出し、必要な列のみ表示
df_cc_crt[df_cc_crt["ccid"] == "C90714"][["ccname", "mcname", "crtname"]]

,ccname,mcname,crtname
2837,TheThreeWoodCreation殺人事件,週刊少年マガジン,しもさか保
2838,TheThreeWoodCreation殺人事件,週刊少年マガジン,ちばてつや
2839,TheThreeWoodCreation殺人事件,週刊少年マガジン,もとはしまさひで
2840,TheThreeWoodCreation殺人事件,週刊少年マガジン,三浦みつる
2841,TheThreeWoodCreation殺人事件,週刊少年マガジン,井上大助
2842,TheThreeWoodCreation殺人事件,週刊少年マガジン,大和田夏希
2843,TheThreeWoodCreation殺人事件,週刊少年マガジン,小林まこと
2844,TheThreeWoodCreation殺人事件,週刊少年マガジン,川三番地
2845,TheThreeWoodCreation殺人事件,週刊少年マガジン,服部かずみ
2846,TheThreeWoodCreation殺人事件,週刊少年マガジン,本山一城


### `crtid`、`crtname`列の深掘り

`crtid`（マンガ作者ID）と`crtname`（マンガ作者名）について分析してみましょう。

まずは、`crtid`と`crtname`の対応関係を確認します。

In [69]:
# `crtid`単位で`crtname`のユニーク数を集計し、その統計情報を取得
df_cc_crt.groupby("crtid")["crtname"].nunique().describe().reset_index()

,index,crtname
0,count,3146.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


一つの`crtid`に対して、必ず一つの`crtname`が紐付いていることが確認できました。
では、その逆はどうでしょうか。

In [70]:
# `crtname`単位で`crtid`のユニーク数を集計し、その統計情報を取得
df_cc_crt.groupby("crtname")["crtid"].nunique().describe().reset_index()

,index,crtid
0,count,3146.0
1,mean,1.0
2,std,0.0
3,min,1.0
4,25%,1.0
5,50%,1.0
6,75%,1.0
7,max,1.0


逆についても同様でした。つまり、`crtid`と`crtname`が一対一で対応付けられていると言えます。

次に、一人のマンガ作者あたり、どの程度のマンガ作品と紐付いているか分析してみます。

In [71]:
# `crtid`および`crtname`単位で、紐付けられている`ccid`のユニーク数を集計
# その結果の基礎統計情報を取得
df_cc_crt.groupby(["crtid", "crtname"])["ccid"].nunique().describe().reset_index()

,index,ccid
0,count,3146.000000
1,mean,2.513986
2,std,3.246516
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,3.000000
7,max,54.000000


一人のマンガ家に対し、平均で約2.5、最大で54のマンガ作品が紐付けられていることがわかります。
`ccid`数が多い順に`crtname`を集計してみます。

In [72]:
# `crtname`単位で、紐付けられている`ccid`のユニーク数を集計
# 結果を降順にソートし、上位10名のマンガ作者とその紐づくマンガ作品数を抽出
df_cc_crt.groupby("crtname")["ccid"].nunique().sort_values(
    ascending=False
).reset_index().head(10)

,crtname,ccid
0,水島新司,54
1,手塚治虫,47
2,松本零士,39
3,ジョージ秋山,37
4,石森章太郎,31
5,三枝義浩,27
6,鳥山明,26
7,本宮ひろ志,26
8,梶原一騎,25
9,小山田いく,25


`水島新司`さんが最も多く、54の`ccid`と紐付けられていることがわかります。

理解を深めるため、25以上の`ccid`を紐付けられているマンガ作者について、その作品の各話数に関する基礎集計を行います。

In [73]:
# `crtname`単位で、紐づいている`ccid`のユニーク数を集計して中間テーブルを作成
df_tmp = df_cc_crt.groupby("crtname")["ccid"].nunique().reset_index()

# `ccid`の数が25以上のマンガ作者の名前リストを作成
crtnames = df_tmp[df_tmp["ccid"] >= 25]["crtname"].unique()

# 上記で取得したマンガ作者に関して、`n_ce`の基礎統計量を計算し、件数で降順ソート
df_cc_crt[df_cc_crt["crtname"].isin(crtnames)].groupby("crtname")[
    "n_ce"
].describe().sort_values(by="count", ascending=False)

,count,mean,std,min,25%,50%,75%,max
crtname,,,,,,,,
水島新司,54.0,51.814815,118.527614,1.0,1.0,1.0,21.75,635.0
手塚治虫,47.0,21.021277,46.156864,1.0,1.0,2.0,19.50,257.0
松本零士,39.0,6.282051,19.740352,1.0,1.0,1.0,1.00,115.0
ジョージ秋山,37.0,23.648649,35.246920,1.0,1.0,12.0,33.00,198.0
石森章太郎,31.0,23.870968,25.699211,1.0,1.0,23.0,33.00,100.0
三枝義浩,27.0,3.037037,6.601174,1.0,2.0,2.0,2.00,36.0
本宮ひろ志,26.0,20.384615,30.415887,1.0,1.0,1.5,28.75,106.0
鳥山明,26.0,31.961538,109.568784,1.0,1.0,1.0,2.75,519.0
小山田いく,25.0,25.080000,34.264802,1.0,1.0,5.0,44.00,102.0


今回は、分布の形状を表す四分位値（`25%`、`50%`、`75%`）に注目してみましょう。多くの`ccid`と紐付けられたマンガ作者は、いわゆる読切作品を多産しているため、`50%`付近まで`n_ce`が非常に小さいことがわかります。
そこで、一話完結の読切作品を除外し、再集計してみましょう。

In [74]:
# 一話完結を除き、`crtname`単位で紐づいている`ccid`のユニーク数を集計
# 上位10人のマンガ作者と、それに紐づくマンガ作品数を降順ソートで表示
df_cc_crt[df_cc_crt["n_ce"] > 1].groupby("crtname")["ccid"].nunique().sort_values(
    ascending=False
).reset_index().head(10)

,crtname,ccid
0,ジョージ秋山,25
1,手塚治虫,25
2,梶原一騎,24
3,石森章太郎,22
4,三枝義浩,21
5,水島新司,19
6,小山田いく,16
7,赤塚不二夫,15
8,牛次郎,15
9,高橋よしひろ,14


一話完結作品を除くと、`ジョージ秋山`さんと`手塚治虫`さんが最も多くのマンガ作品と紐付いていることがわかります。

### `n_ce`列の深掘り

`n_ce`（マンガ各話数）について分析してみましょう。

まずは、マンガ作者あたりの`n_ce`について基礎統計量を確認します。

In [75]:
# `crtname`単位での`n_ce`の合計を集計
# その集計結果の基礎統計量を算出
df_cc_crt.groupby("crtname")["n_ce"].sum().describe().reset_index()

,index,n_ce
0,count,3146.000000
1,mean,66.833121
2,std,165.971265
3,min,1.000000
4,25%,1.000000
5,50%,8.000000
6,75%,41.750000
7,max,2798.000000


一人のマンガ作者あたり、平均約67話のマンガ各話と紐付けられていることがわかります。

それでは、合計`n_ce`が多いマンガ作者を抽出してみましょう。

In [76]:
# `crtname`単位での`n_ce`の合計を集計し、降順にソート
# 上位10人のマンガ作者を抽出
df_cc_crt.groupby("crtname")["n_ce"].sum().sort_values(
    ascending=False
).reset_index().head(10)

,crtname,n_ce
0,水島新司,2798
1,秋本治,1979
2,梶原一騎,1873
3,高橋留美子,1736
4,浜岡賢次,1384
5,青山剛昌,1288
6,あだち充,1269
7,森川ジョージ,1226
8,満田拓也,1223
9,板垣恵介,1194


`水島新司`さんは`n_ce`数でも圧倒的であることがわかりました。
先程の、`ccid`による集計値と顔ぶれが変わるのは非常に興味深いです。

### `n_2c`、`n_4c`列の深掘り

`n_2c`（マンガ作品の合計2色カラー話数）と`n_4c`（マンガ作品の合計4色カラー話数）について分析してみましょう。

まず、マンガ作者あたりの`n_2c`および`n_4c`について基礎統計量を算出します。

In [77]:
# `crtname`単位での`n_2c`および`n_4c`の合計を集計し、基礎統計量を算出
df_cc_crt.groupby("crtname")[["n_2c", "n_4c"]].sum().describe().reset_index()

,index,n_2c,n_4c
0,count,3146.000000,3146.000000
1,mean,2.517483,4.835982
2,std,10.900118,16.289349
3,min,0.000000,0.000000
4,25%,0.000000,0.000000
5,50%,0.000000,1.000000
6,75%,0.000000,3.000000
7,max,281.000000,491.000000


一人のマンガ作者あたり、平均約3話の2色カラー各話と、平均約5話の4色カラー各話と紐付けられていることがわかります。
では、特に多くの2色カラー各話を掲載したマンガ作者を抽出してみましょう。

In [78]:
# `crtname`単位での`n_2c`の合計を集計し、降順にソート
# 上位5人のマンガ作者を抽出
df_cc_crt.groupby("crtname")["n_2c"].sum().sort_values(
    ascending=False
).reset_index().head()

,crtname,n_2c
0,水島新司,281
1,山上たつひこ,206
2,柳沢きみお,113
3,鳥山明,98
4,石渡治,93


最も多くの2色カラー各話を掲載したのは、`水島新司`さんであることがわかりました。それでは、4色カラーはどうでしょうか？

In [79]:
# `crtname`単位での`n_4c`の合計を集計し、降順にソート
# 上位5人のマンガ作者を抽出
df_cc_crt.groupby("crtname")["n_4c"].sum().sort_values(
    ascending=False
).reset_index().head()

,crtname,n_4c
0,水島新司,491
1,板垣恵介,281
2,尾田栄一郎,160
3,渡辺航,150
4,浜岡賢次,148


4色カラーに関しても`水島新司`さんが圧倒的であることがわかりました。

### `first_date`、`last_date`列の深掘り

`first_date`（マンガ作品の最初の公開日）と`last_date`（マンガ作品の最後の公開日）について分析してみましょう。

まず、分析用に`first_date`と`last_date`の型を変更します。

In [80]:
# `first_date`列をdatetime型に変換
df_cc_crt["first_date"] = pd.to_datetime(df_cc_crt["first_date"])
# `last_date`列をdatetime型に変換
df_cc_crt["last_date"] = pd.to_datetime(df_cc_crt["last_date"])

マンガ作者あたりの活動期間の長さ（最も古い`first_date`から最も新しい`last_date`までの日数）を集計します。

In [81]:
# マンガ作者ごとに最初と最後の活動日を集計
df_tmp = (
    df_cc_crt.groupby("crtname")[["first_date", "last_date"]]
    .agg({"first_date": "min", "last_date": "max"})
    .reset_index()
)

# 各マンガ作者の活動期間を算出
df_tmp["duration"] = df_tmp["last_date"] - df_tmp["first_date"]

# 活動期間が長い上位10名のマンガ作者を取得
df_tmp.sort_values(by="duration", ascending=False).head(10)

,crtname,first_date,last_date,duration
2144,水島新司,1970-07-27,2017-07-13,17153 days
1701,手塚治虫,1970-07-27,2013-05-16,15634 days
2160,永井豪,1970-07-27,2012-09-27,15403 days
2025,梶原一騎,1970-07-27,2011-01-15,14782 days
589,三浦みつる,1972-12-18,2013-05-09,14752 days
225,つのだじろう,1971-02-22,2011-06-16,14724 days
2535,秋本治,1976-07-19,2016-10-03,14686 days
3061,高橋留美子,1978-07-09,2017-07-12,14248 days
2815,車田正美,1978-03-06,2017-02-02,14213 days
465,ジョージ秋山,1970-07-27,2009-06-18,14206 days


このデータにおいて、最も長い期間マンガ作品を公開しているのは、`水島新司`さんでした。

ただし、（他の集計結果にも言えることですが）これはこのデータの対象期間・対象マンガ雑誌における活動期間であることに注意が必要です。
例えば`水島新司`さんは1958年に、`手塚治虫`さんは1946年にデビューしたとされていますが、それらの情報は本書のデータの対象外です。

### `mcid`、`mcname`列の深掘り

`mcid`（マンガ雑誌ID）と`mcname`（マンガ雑誌名）について分析してみましょう。

まず、マンガ作者あたりのマンガ雑誌数について集計します。

In [82]:
# マンガ作者ごとに活動したマンガ雑誌数を集計
df_cc_crt.groupby("crtname")["mcname"].nunique().describe().reset_index()

,index,mcname
0,count,3146.000000
1,mean,1.119517
2,std,0.426865
3,min,1.000000
4,25%,1.000000
5,50%,1.000000
6,75%,1.000000
7,max,4.000000


一人のマンガ作者は、（四大少年誌に限れば）一つのマンガ雑誌と紐付けられることが多いことがわかります。

それでは、四大少年誌全てに掲載経験のあるマンガ作者はどの程度いるのでしょうか？

In [83]:
# マンガ作者ごとに活動したマンガ雑誌IDの数を集計
df_tmp = df_cc_crt.groupby("crtname")["mcid"].nunique().reset_index()
# 4つの主要な少年誌すべてに掲載経験があるマンガ作者を特定
crtnames = df_tmp[df_tmp["mcid"] == 4]["crtname"].unique()
# 特定したマンガ作者に関して、各マンガ雑誌での掲載回数を集計
df_crt_mcname = (
    df_cc_crt[df_cc_crt["crtname"].isin(crtnames)]
    .groupby(["crtname", "mcname"])["n_ce"]
    .sum()
    .reset_index()
)
df_crt_mcname

,crtname,mcname,n_ce
0,ジョージ秋山,週刊少年サンデー,148
1,ジョージ秋山,週刊少年ジャンプ,147
2,ジョージ秋山,週刊少年チャンピオン,369
3,ジョージ秋山,週刊少年マガジン,211
4,ビッグ錠,週刊少年サンデー,22
...,...,...,...
91,赤塚不二夫,週刊少年マガジン,276
92,鈴木央,週刊少年サンデー,223
93,鈴木央,週刊少年ジャンプ,181
94,鈴木央,週刊少年チャンピオン,17


想像より多くのマンガ作者が、四大少年誌全てで掲載経験がありました。

In [84]:
# 四大少年誌全てに掲載経験のあるマンガ作者一覧
crtnames

array(['ジョージ秋山', 'ビッグ錠', '小林よしのり', '手塚治虫', '望月三起也', '本宮ひろ志', '村生ミオ',
       '村野守美', '松本零士', '柳沢きみお', '梅本さちお', '梶原一騎', '水木しげる', '永井豪', '池内誠一',
       '滝沢解', '牛次郎', '真樹日佐夫', '石森章太郎', '篠原とおる', '芳谷圭児', '貝塚ひろし', '赤塚不二夫',
       '鈴木央'], dtype=object)

参考までに、合計`n_ce`数の多い上位5名に対してピボットテーブルを表示します。

In [85]:
# n_ceの総数に基づいて、上位5名のマンガ作者を特定
crtnames = (
    df_crt_mcname.groupby("crtname")["n_ce"]
    .sum()
    .sort_values(ascending=False)
    .index[:5]
)
# 上位5名のマンガ作者に関して、マンガ雑誌別の掲載回数をピボットテーブルとして表示
df_crt_mcname[df_crt_mcname["crtname"].isin(crtnames)].pivot_table(
    index="crtname", columns="mcname", values="n_ce"
)

mcname,週刊少年サンデー,週刊少年ジャンプ,週刊少年チャンピオン,週刊少年マガジン
crtname,,,,
ジョージ秋山,148.0,147.0,369.0,211.0
手塚治虫,43.0,27.0,725.0,193.0
柳沢きみお,21.0,109.0,463.0,223.0
梶原一騎,545.0,215.0,159.0,954.0
石森章太郎,400.0,4.0,120.0,216.0


それぞれのマンガ作者ごとに、特徴的な結果が得られました。

- `ジョージ秋山`さん：`週刊少年チャンピオン`、`週刊少年マガジン`を中心に掲載
- `手塚治虫`さん：`週刊少年チャンピオン`を中心に掲載
- `柳沢みきお`さん：`週刊少年チャンピオン`、`週刊少年マガジン`を中心に掲載
- `梶原一騎`さん：`週刊少年マガジン`、`週刊少年サンデー`を中心に掲載
- `石ノ森章太郎`さん：`週刊少年サンデー`、`週刊少年マガジン`を中心に掲載

## まとめ

ここでは、`cm_ce.csv`および`cm_cc_crt.csv`について基礎分析を行いました。
重要な発見や示唆がたくさんありますので、もう一度復習しておきましょう。

まず大前提として、本書で扱うマンガデータは、
[MADB Lab v1.0](https://github.com/mediaarts-db/dataset/tree/1.0)として公開されているものをGitHub経由で取得し、[所定の前処理](../appendix/cm_preprocess.ipynb)を行ったものです。
発行日が **1970年7月27日から2017年7月31日まで** の **四大少年誌（週刊少年サンデー、週刊少年ジャンプ、週刊少年チャンピオン、そして週刊少年マガジン）** を対象としています。
特に、 **上記の対象外で活躍したマンガ作者が過小に評価され得る** という点は、常に意識しておくべきです。

`cm_ce.csv`は、マンガ各話に関する情報をまとめたファイルです。
今後分析で用いる際は、以下に特に注意するべきでしょう：

- `price`および`cename`に一定程度の欠損がある
- `miid`と`miname`が一対一に対応していないケースが少数存在する
    - 例：`週刊少年ジャンプ 2012年 表示号数18`。同一`miname`に複数の`miid`が紐づいている。これは裏表紙を表紙として読めるような特殊な印刷をした号であり、表紙側から読める作品を`M616417`と紐づけ、裏表紙側から読める作品を`M616418`と紐づけている
- `ccid`と`ccname`が一対一に対応していないケースが少数存在する
    - 例：`MAJOR`。経緯は不明だが、2000年付近を境に`ccid`が変更されている
- ユニークな`ceid`数（合計各話数）が非常に少ない`ccid`が存在する
    - 例：`'80オリンピックに燃える日本のホープ!!`、`'83打ち上げ!!爆笑ギャグ・バトル`等
    - 本書では、 **合計各話数が8以上のマンガ作品** を連載作品として扱う
- `pages`が非常に大きいケースが少数存在する
    - 例：`週刊少年ジャンプ 2015年 表示号数48`の`磯部磯兵衛物語～浮世はつらいよ～`。巻号一冊中に二話が掲載された結果、一話目の`page_start`と二話目の`page_end`が大きく離れてしまい、`pages`が例外的に大きくなった可能性がある
- `price`が非常に大きいケースが少数存在する
    - 例：`週刊少年サンデー 2007年 表示号数50`。`price`の最大値`371`円だが、この値の正誤含め原因は不明

`cm_cc_crt.csv`は、マンガ作品とマンガ作者の関係をまとめたファイルです。
 今後分析で用いる際は、以下に特に注意するべきでしょう：
 
- `ccid`と`crtid`の組合せが主キーとなっているため、それぞれ単独では重複する行が存在する
- `cm_ce.csv`に存在する`ccid`のうち、約98%にあたる6844の`ccid`をカバー
- `first_date`および`last_date`は、あくまでもこのデータの対象期間中の最初と最後の掲載日